# Numerical Differentiation Methods

This notebook demonstrates various numerical differentiation techniques in rustlab-numerical, including finite differences, Richardson extrapolation, and advanced methods for computing derivatives with high accuracy.

## Setup and Dependencies

In [2]:
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-numerical = { path = ".." }
:dep rustlab-plotting = { path = "../../rustlab-plotting" }
:dep num-complex = "0.4"

use rustlab_math::{VectorF64, vec64, range, FunctionalMap};
use rustlab_numerical::differentiation::*;
use rustlab_plotting::*;
use std::f64::consts::PI;
use num_complex::Complex64;

## 1. Forward Differences

Forward differences approximate derivatives using f'(x) ≈ (f(x+h) - f(x))/h with various orders of accuracy.

In [3]:
{
    // Test forward differences on f(x) = x² (derivative = 2x)
    let f = |x: f64| x * x;
    let x_test = 2.0;
    let exact_derivative = 2.0 * x_test; // f'(2) = 4
    
    println!("Forward Difference Analysis for f(x) = x² at x = {}", x_test);
    println!("Exact derivative f'({}) = {:.6}", x_test, exact_derivative);
    println!();
    println!("{:<12} {:<8} {:<15} {:<15} {:<15}", "Step Size", "Order", "Approximation", "Error", "Error Ratio");
    
    let step_sizes = vec![1e-2, 1e-3, 1e-4, 1e-5];
    
    // Test order 1 and order 2 forward differences
    for order in &[1, 2] {
        println!("\nOrder {} accuracy:", order);
        let mut prev_error = 0.0;
        
        for &h in &step_sizes {
            let forward_deriv = forward_diff(f, x_test, h, *order)?;
            let error = (forward_deriv - exact_derivative).abs();
            let ratio = if prev_error > 0.0 { prev_error / error } else { 0.0 };
            
            println!("{:<12.0e} {:<8} {:<15.8} {:<15.2e} {:<15.2}", h, order, forward_deriv, error, ratio);
            prev_error = error;
        }
    }
}

Forward Difference Analysis for f(x) = x² at x = 2
Exact derivative f'(2) = 4.000000

Step Size    Order    Approximation   Error           Error Ratio    

Order 1 accuracy:
1e-2         1        4.01000000      1.00e-2         0.00           
1e-3         1        4.00100000      1.00e-3         10.00          
1e-4         1        4.00010000      1.00e-4         10.00          
1e-5         1        4.00001000      1.00e-5         10.00          

Order 2 accuracy:
1e-2         2        4.00000000      2.18e-13        0.00           
1e-3         2        4.00000000      3.55e-15        61.50          
1e-4         2        4.00000000      1.73e-11        0.00           
1e-5         2        4.00000000      2.62e-11        0.66           


()

## 2. Backward Differences

Backward differences use f'(x) ≈ (f(x) - f(x-h))/h, with various orders of accuracy.

In [4]:
{
    // Compare backward differences with forward differences
    let f = |x: f64| x * x;
    let x_test = 2.0;
    let exact_derivative = 4.0;
    
    println!("Backward vs Forward Differences Comparison (Order 1)");
    println!("{:<12} {:<15} {:<15} {:<15} {:<15}", "Step Size", "Forward", "Backward", "Forward Error", "Backward Error");
    
    let step_sizes = vec![1e-2, 1e-3, 1e-4, 1e-5];
    
    for h in step_sizes {
        let forward_deriv = forward_diff(f, x_test, h, 1)?;  // Order 1
        let backward_deriv = backward_diff(f, x_test, h, 1)?; // Order 1
        
        let forward_error = (forward_deriv - exact_derivative).abs();
        let backward_error = (backward_deriv - exact_derivative).abs();
        
        println!("{:<12.0e} {:<15.8} {:<15.8} {:<15.2e} {:<15.2e}", 
                 h, forward_deriv, backward_deriv, forward_error, backward_error);
    }
}

Backward vs Forward Differences Comparison (Order 1)
Step Size    Forward         Backward        Forward Error   Backward Error 
1e-2         4.01000000      3.99000000      1.00e-2         1.00e-2        
1e-3         4.00100000      3.99900000      1.00e-3         1.00e-3        
1e-4         4.00010000      3.99990000      1.00e-4         1.00e-4        
1e-5         4.00001000      3.99999000      1.00e-5         1.00e-5        


()

## 3. Central Differences

Central differences use f'(x) ≈ (f(x+h) - f(x-h))/(2h) for better accuracy.

In [5]:
{
    // Demonstrate superior accuracy of central differences
    let f = |x: f64| x.sin(); // f'(x) = cos(x)
    let x_test = PI / 4.0;     // Test at π/4
    let exact_derivative = x_test.cos(); // cos(π/4) = 1/√2
    
    println!("Central Differences vs Forward/Backward for f(x) = sin(x) at x = π/4");
    println!("Exact derivative f'(π/4) = cos(π/4) = {:.8}", exact_derivative);
    println!();
    println!("{:<12} {:<15} {:<15} {:<15}", "Step Size", "Forward Error", "Central Error", "Improvement");
    
    let step_sizes = vec![1e-1, 1e-2, 1e-3, 1e-4, 1e-5];
    
    for h in step_sizes {
        let forward_deriv = forward_diff(f, x_test, h, 1)?;  // Order 1 forward
        let central_deriv = central_diff(f, x_test, h, 2)?;  // Order 2 central
        
        let forward_error = (forward_deriv - exact_derivative).abs();
        let central_error = (central_deriv - exact_derivative).abs();
        let improvement = forward_error / central_error;
        
        println!("{:<12.0e} {:<15.2e} {:<15.2e} {:<15.1}", 
                 h, forward_error, central_error, improvement);
    }
}

Central Differences vs Forward/Backward for f(x) = sin(x) at x = π/4
Exact derivative f'(π/4) = cos(π/4) = 0.70710678

Step Size    Forward Error   Central Error   Improvement    
1e-1         3.65e-2         1.18e-3         31.0           
1e-2         3.55e-3         1.18e-5         301.0          
1e-3         3.54e-4         1.18e-7         3001.0         
1e-4         3.54e-5         1.18e-9         29999.1        
1e-5         3.54e-6         1.40e-11        253194.8       


()

## 4. Higher-Order Finite Differences

Higher-order schemes provide better accuracy by using more function evaluations.

In [6]:
{
    // Compare different orders of central differences
    let f = |x: f64| x.exp(); // f'(x) = exp(x)
    let x_test: f64 = 1.0;
    let exact_derivative = x_test.exp(); // e¹ = e
    
    println!("Higher-Order Central Differences for f(x) = eˣ at x = 1");
    println!("Exact derivative f'(1) = e = {:.10}", exact_derivative);
    println!();
    println!("{:<12} {:<15} {:<15} {:<15}", "Step Size", "2nd Order", "4th Order", "Error Reduction");
    
    let step_sizes = vec![1e-1, 1e-2, 1e-3, 1e-4];
    
    for h in step_sizes {
        let second_order = central_diff(f, x_test, h, 2)?;  // Order 2
        let fourth_order = central_diff(f, x_test, h, 4)?;  // Order 4
        
        let error_2nd = (second_order - exact_derivative).abs();
        let error_4th = (fourth_order - exact_derivative).abs();
        let reduction = if error_4th > 0.0 { error_2nd / error_4th } else { 0.0 };
        
        println!("{:<12.0e} {:<15.2e} {:<15.2e} {:<15.1}", 
                 h, error_2nd, error_4th, reduction);
    }
}

Higher-Order Central Differences for f(x) = eˣ at x = 1
Exact derivative f'(1) = e = 2.7182818285

Step Size    2nd Order       4th Order       Error Reduction
1e-1         4.53e-3         9.07e-6         499.7          
1e-2         4.53e-5         9.06e-10        49999.7        
1e-3         4.53e-7         4.40e-13        1030475.2      
1e-4         4.53e-9         1.23e-12        3696.4         


()

## 5. Richardson Extrapolation

Richardson extrapolation combines results at different step sizes to achieve higher accuracy.

In [7]:
{
    // Demonstrate Richardson extrapolation for derivative estimation
    let f = |x: f64| (x * x * x).sin(); // Complex function
    let x_test: f64 = 0.5;
    
    // Exact derivative: f'(x) = 3x²cos(x³) 
    let exact_derivative = 3.0 * x_test * x_test * (x_test * x_test * x_test).cos();
    
    println!("Richardson Extrapolation for f(x) = sin(x³) at x = 0.5");
    println!("Exact derivative = {:.10}", exact_derivative);
    println!();
    
    // Richardson extrapolation with central differences
    let base_steps = vec![1e-2, 1e-3, 1e-4, 1e-5];
    
    println!("{:<12} {:<15} {:<15} {:<15}", "Base h", "Standard", "Richardson", "Improvement");
    
    for h in base_steps {
        // Standard central difference
        let standard = central_diff(f, x_test, h, 2)?;
        
        // Richardson extrapolation using correct parameter order
        let richardson = richardson_extrapolation(
            f,
            x_test,
            h,
            |func, x, step, order| central_diff(func, x, step, order),
            2  // Order of the method
        )?;
        
        let standard_error = (standard - exact_derivative).abs();
        let richardson_error = (richardson - exact_derivative).abs();
        let improvement = if richardson_error > 0.0 { standard_error / richardson_error } else { 0.0 };
        
        println!("{:<12.0e} {:<15.2e} {:<15.2e} {:<15.1}", 
                 h, standard_error, richardson_error, improvement);
    }
}

Richardson Extrapolation for f(x) = sin(x³) at x = 0.5
Exact derivative = 0.7441482504

Base h       Standard        Richardson      Improvement    
1e-2         7.82e-5         3.22e-9         24281.8        
1e-3         7.82e-7         2.23e-13        3503327.5      
1e-4         7.82e-9         2.67e-13        29281.3        
1e-5         7.71e-11        5.91e-13        130.5          


()

## 6. Complex-Step Differentiation

Complex-step differentiation can achieve machine precision accuracy for analytic functions.

In [8]:
{
    // Define a function that works with complex numbers
    let f_complex = |z: Complex64| z.exp() * z;
    
    // Wrapper for real function evaluation
    let f_real = |x: f64| x.exp() * x;
    
    let x_test: f64 = 2.0;
    // Exact derivative: d/dx(xe^x) = e^x + xe^x = (1+x)e^x
    let exact_derivative = (1.0 + x_test) * x_test.exp();
    
    println!("Complex-Step Differentiation vs Finite Differences");
    println!("Function: f(x) = x*eˣ at x = 2.0");
    println!("Exact derivative = {:.12}", exact_derivative);
    println!();
    
    // Test different methods
    let h_optimal = 1e-8; // Good step size for central differences
    let central_result = central_diff(f_real, x_test, h_optimal, 4)?; // Higher-order central
    let central_error = (central_result - exact_derivative).abs();
    
    // Complex-step differentiation
    let h_complex = 1e-15;
    let complex_result = complex_step_diff(f_complex, x_test, h_complex)?;
    let complex_error = (complex_result - exact_derivative).abs();
    
    println!("Central diff (h={:.0e}, order=4): {:.12} (error: {:.2e})", h_optimal, central_result, central_error);
    println!("Complex-step diff (h={:.0e}):     {:.12} (error: {:.2e})", h_complex, complex_result, complex_error);
    
    if complex_error > 0.0 {
        println!("Accuracy improvement: {:.1}x", central_error / complex_error);
    } else {
        println!("Complex-step achieved machine precision!");
    }
}

Complex-Step Differentiation vs Finite Differences
Function: f(x) = x*eˣ at x = 2.0
Exact derivative = 22.167168296792

Central diff (h=1e-8, order=4): 22.167168027091 (error: 2.70e-7)
Complex-step diff (h=1e-15):     22.167168296792 (error: 0.00e0)
Complex-step achieved machine precision!


()

## 7. Second Derivatives

Numerical computation of second derivatives using finite differences.

In [9]:
{
    // Second derivative using central differences: f''(x) ≈ (f(x+h) - 2f(x) + f(x-h))/h²
    let f = |x: f64| x.powi(4); // f(x) = x⁴, f''(x) = 12x²
    let x_test = 1.5;
    let exact_second_deriv = 12.0 * x_test * x_test; // f''(1.5) = 12 * 2.25 = 27
    
    println!("Second Derivative Computation for f(x) = x⁴ at x = 1.5");
    println!("Exact second derivative f''(1.5) = {:.6}", exact_second_deriv);
    println!();
    println!("{:<12} {:<15} {:<15} {:<15}", "Step Size", "Approximation", "Error", "Error Ratio");
    
    let step_sizes = vec![1e-1, 1e-2, 1e-3, 1e-4, 1e-5];
    let mut prev_error = 0.0;
    
    for h in step_sizes {
        // Compute second derivative using three-point formula
        let f_plus = f(x_test + h);
        let f_center = f(x_test);
        let f_minus = f(x_test - h);
        let second_deriv = (f_plus - 2.0 * f_center + f_minus) / (h * h);
        
        let error = (second_deriv - exact_second_deriv).abs();
        let ratio = if prev_error > 0.0 { prev_error / error } else { 0.0 };
        
        println!("{:<12.0e} {:<15.8} {:<15.2e} {:<15.2}", h, second_deriv, error, ratio);
        prev_error = error;
    }
}

Second Derivative Computation for f(x) = x⁴ at x = 1.5
Exact second derivative f''(1.5) = 27.000000

Step Size    Approximation   Error           Error Ratio    
1e-1         27.02000000     2.00e-2         0.00           
1e-2         27.00020000     2.00e-4         100.00         
1e-3         27.00000200     2.00e-6         99.94          
1e-4         27.00000019     1.91e-7         10.47          
1e-5         26.99999335     6.65e-6         0.03           


()

## 8. Gradient Computation

Computing gradients for multivariate functions using finite differences.

In [10]:
{
    // Example: f(x,y) = x²y + xy² = xy(x+y)
    // ∇f = (∂f/∂x, ∂f/∂y) = (2xy + y², x² + 2xy)
    let f = |coords: &[f64]| -> f64 {
        let x = coords[0];
        let y = coords[1];
        x * x * y + x * y * y
    };
    
    let point = vec![2.0, 3.0]; // Test at (2, 3)
    let x = point[0];
    let y = point[1];
    
    // Exact gradient at (2, 3)
    let exact_grad_x = 2.0 * x * y + y * y; // 2*2*3 + 3² = 12 + 9 = 21
    let exact_grad_y = x * x + 2.0 * x * y; // 2² + 2*2*3 = 4 + 12 = 16
    
    println!("Gradient Computation for f(x,y) = x²y + xy² at (2, 3)");
    println!("Exact gradient: (∂f/∂x, ∂f/∂y) = ({}, {})", exact_grad_x, exact_grad_y);
    println!();
    
    // Compute numerical gradient using central differences
    let h = 1e-8;
    let mut gradient = vec![0.0; 2];
    
    // Partial derivative with respect to x
    let mut point_plus = point.clone();
    let mut point_minus = point.clone();
    
    point_plus[0] += h;
    point_minus[0] -= h;
    gradient[0] = (f(&point_plus) - f(&point_minus)) / (2.0 * h);
    
    // Partial derivative with respect to y
    point_plus = point.clone();
    point_minus = point.clone();
    point_plus[1] += h;
    point_minus[1] -= h;
    gradient[1] = (f(&point_plus) - f(&point_minus)) / (2.0 * h);
    
    let error_x = (gradient[0] - exact_grad_x).abs();
    let error_y = (gradient[1] - exact_grad_y).abs();
    
    println!("Numerical gradient: ({:.8}, {:.8})", gradient[0], gradient[1]);
    println!("Errors: (∂f/∂x: {:.2e}, ∂f/∂y: {:.2e})", error_x, error_y);
    
    println!("\nFunction value at test point: f(2,3) = {:.2}", f(&point));
    println!("Gradient magnitude: {:.4}", (gradient[0].powi(2) + gradient[1].powi(2)).sqrt());
}

Gradient Computation for f(x,y) = x²y + xy² at (2, 3)
Exact gradient: (∂f/∂x, ∂f/∂y) = (21, 16)

Numerical gradient: (20.99999996, 15.99999990)
Errors: (∂f/∂x: 3.88e-8, ∂f/∂y: 9.72e-8)

Function value at test point: f(2,3) = 30.00
Gradient magnitude: 26.4008


()

## 9. Optimal Step Size Analysis

Understanding the trade-off between truncation and round-off errors.

In [11]:
{
    // Demonstrate truncation vs round-off error trade-off
    let f = |x: f64| x.sin();
    let x_test: f64 = 1.0;
    let exact_derivative = x_test.cos();
    
    // Test a wide range of step sizes
    let step_powers: Vec<i32> = (-16..=0).collect();
    let mut step_sizes = Vec::new();
    let mut errors = Vec::new();
    
    println!("Error Analysis: Truncation vs Round-off for f(x) = sin(x) at x = 1");
    println!("Exact derivative = {:.12}", exact_derivative);
    println!();
    println!("{:<12} {:<15} {:<15}", "Step Size", "Approximation", "Error");
    
    for &power in &step_powers {
        let h = 10.0_f64.powi(power);
        let approx_deriv = central_diff(f, x_test, h, 2)?;
        let error = (approx_deriv - exact_derivative).abs();
        
        step_sizes.push(h);
        errors.push(error);
        
        if power >= -8 && power <= 0 { // Print only a subset for clarity
            println!("{:<12.0e} {:<15.10} {:<15.2e}", h, approx_deriv, error);
        }
    }
    
    // Find optimal step size (minimum error)
    let min_error_idx = errors.iter()
        .enumerate()
        .min_by(|(_, a), (_, b)| a.partial_cmp(b).unwrap())
        .unwrap().0;
    
    let optimal_h = step_sizes[min_error_idx];
    let optimal_error = errors[min_error_idx];
    
    println!();
    println!("Optimal step size: h = {:.0e}", optimal_h);
    println!("Minimum error: {:.2e}", optimal_error);
    
    // Plot error vs step size
    let step_vec = VectorF64::from_slice(&step_sizes);
    let error_vec = VectorF64::from_slice(&errors);
    
    Plot::new()
        .line_with(&step_vec, &error_vec, "Differentiation Error")
        .title("Error vs Step Size for Central Differences")
        .xlabel("Step Size h")
        .ylabel("Absolute Error")
        .xscale(Scale::Log10)
        .yscale(Scale::Log10)
        .grid(true)
        .show()?;
    
    println!("Error analysis plot generated!");
}

Error Analysis: Truncation vs Round-off for f(x) = sin(x) at x = 1
Exact derivative = 0.540302305868

Step Size    Approximation   Error          
1e-8         0.5403023084    2.58e-9        
1e-7         0.5403023057    1.94e-10       
1e-6         0.5403023059    2.77e-11       
1e-5         0.5403023059    1.11e-11       
1e-4         0.5403023050    9.00e-10       
1e-3         0.5403022158    9.01e-8        
1e-2         0.5402933009    9.00e-6        
1e-1         0.5394022522    9.00e-4        
1e0          0.4546487134    8.57e-2        

Optimal step size: h = 1e-5
Minimum error: 1.11e-11
Error analysis plot generated!


()

## 10. Performance Comparison

Benchmarking different differentiation methods for computational efficiency.

In [12]:
{
    use std::time::Instant;
    
    let f = |x: f64| (x * x + 1.0).sqrt().sin(); // Complex function
    let x_test = 1.0;
    let h = 1e-8;
    let n_iterations = 100000;
    
    println!("Performance Benchmark ({} iterations)", n_iterations);
    println!("Function: f(x) = sin(√(x² + 1)) at x = 1.0");
    println!();
    
    // Forward difference benchmark
    let start = Instant::now();
    for _ in 0..n_iterations {
        let _ = forward_diff(f, x_test, h, 1).unwrap();
    }
    let forward_time = start.elapsed();
    
    // Central difference benchmark
    let start = Instant::now();
    for _ in 0..n_iterations {
        let _ = central_diff(f, x_test, h, 2).unwrap();
    }
    let central_time = start.elapsed();
    
    // Higher-order central difference
    let start = Instant::now();
    for _ in 0..n_iterations {
        let _ = central_diff(f, x_test, h, 4).unwrap();
    }
    let central4_time = start.elapsed();
    
    println!("{:<20} {:<15} {:<15}", "Method", "Total Time", "ns/call");
    println!("{:<20} {:<15.2?} {:<15.1}", "Forward (O1)", forward_time, 
             forward_time.as_nanos() as f64 / n_iterations as f64);
    println!("{:<20} {:<15.2?} {:<15.1}", "Central (O2)", central_time,
             central_time.as_nanos() as f64 / n_iterations as f64);
    println!("{:<20} {:<15.2?} {:<15.1}", "Central (O4)", central4_time,
             central4_time.as_nanos() as f64 / n_iterations as f64);
    
    println!();
    println!("Speed ratios (relative to forward difference):");
    println!("Central O2: {:.2}x", forward_time.as_nanos() as f64 / central_time.as_nanos() as f64);
    println!("Central O4: {:.2}x", forward_time.as_nanos() as f64 / central4_time.as_nanos() as f64);
}

Performance Benchmark (100000 iterations)
Function: f(x) = sin(√(x² + 1)) at x = 1.0

Method               Total Time      ns/call        
Forward (O1)         2.66ms          26.6           
Central (O2)         2.80ms          28.0           
Central (O4)         4.75ms          47.5           

Speed ratios (relative to forward difference):
Central O2: 0.95x
Central O4: 0.56x


()

## 11. Practical Applications

Real-world examples where numerical differentiation is essential.

In [13]:
{
    // Application: Finding critical points of a function
    let f = |x: f64| x.powi(3) - 3.0 * x.powi(2) + 2.0 * x + 1.0;
    
    println!("Finding Critical Points");
    println!("Function: f(x) = x³ - 3x² + 2x + 1");
    println!();
    
    // Search for critical points by finding where f'(x) ≈ 0
    let x_range = range!(-1.0 => 3.0, 100);
    let mut critical_points = Vec::new();
    
    for i in 0..x_range.len() {
        let x = x_range.get(i).unwrap();
        let derivative = central_diff(f, x, 1e-8, 2)?;
        
        // Look for sign changes (simple critical point detection)
        if i > 0 {
            let prev_x = x_range.get(i-1).unwrap();
            let prev_deriv = central_diff(f, prev_x, 1e-8, 2)?;
            
            if derivative * prev_deriv < 0.0 {
                // Sign change detected - refine using bisection
                let mut left = prev_x;
                let mut right = x;
                
                // Simple bisection to refine critical point
                for _ in 0..10 {
                    let mid = (left + right) / 2.0;
                    let mid_deriv = central_diff(f, mid, 1e-8, 2)?;
                    
                    if mid_deriv.abs() < 1e-6 {
                        break;
                    }
                    
                    if mid_deriv * prev_deriv < 0.0 {
                        right = mid;
                    } else {
                        left = mid;
                    }
                }
                
                let critical_x = (left + right) / 2.0;
                critical_points.push(critical_x);
            }
        }
    }
    
    println!("Critical points found:");
    for &cp in &critical_points {
        let value = f(cp);
        // Use finite differences to compute second derivative
        let h = 1e-5;
        let f_plus = f(cp + h);
        let f_center = f(cp);
        let f_minus = f(cp - h);
        let second_deriv = (f_plus - 2.0 * f_center + f_minus) / (h * h);
        
        let nature = if second_deriv > 0.0 { "minimum" } 
                    else if second_deriv < 0.0 { "maximum" }
                    else { "inflection" };
        
        println!("x = {:.4}, f(x) = {:.4}, f''(x) = {:.4} ({})", cp, value, second_deriv, nature);
    }
    
    // Verify with analytical derivative
    println!("\nVerification: f'(x) = 3x² - 6x + 2");
    println!("Critical points occur when 3x² - 6x + 2 = 0");
    println!("Using quadratic formula: x = (6 ± √12) / 6");
    let x1 = (6.0 - 12.0_f64.sqrt()) / 6.0;
    let x2 = (6.0 + 12.0_f64.sqrt()) / 6.0;
    println!("Analytical: x₁ = {:.4}, x₂ = {:.4}", x1, x2);
}

Finding Critical Points
Function: f(x) = x³ - 3x² + 2x + 1

Critical points found:
x = 0.4226, f(x) = 1.3849, f''(x) = -3.4641 (maximum)
x = 1.5774, f(x) = 0.6151, f''(x) = 3.4641 (minimum)

Verification: f'(x) = 3x² - 6x + 2
Critical points occur when 3x² - 6x + 2 = 0
Using quadratic formula: x = (6 ± √12) / 6
Analytical: x₁ = 0.4226, x₂ = 1.5774


()

## Summary

This notebook demonstrated the comprehensive numerical differentiation capabilities of rustlab-numerical:

### Key Takeaways:

1. **Forward/Backward Differences**: Simple methods with O(h) to O(h⁴) accuracy
2. **Central Differences**: Better accuracy with O(h²) to O(h⁶) convergence  
3. **Higher-Order Methods**: Trading more function evaluations for better accuracy
4. **Richardson Extrapolation**: Combining approximations for enhanced precision
5. **Complex-Step Differentiation**: Machine precision for analytic functions
6. **Second Derivatives**: Essential for optimization and curvature analysis
7. **Gradient Computation**: Multivariate function differentiation
8. **Error Analysis**: Understanding truncation vs round-off trade-offs
9. **Performance**: Speed comparisons between methods
10. **Applications**: Critical point finding and optimization

### Best Practices:

1. **Use central differences** for general-purpose differentiation (order 2 or 4)
2. **Apply Richardson extrapolation** when high accuracy is needed
3. **Consider complex-step methods** for maximum precision with analytic functions
4. **Choose step size carefully** - typically h ≈ ε^(1/3) for central differences where ε is machine epsilon
5. **Monitor numerical stability** especially for higher-order methods
6. **Validate results** against analytical derivatives when available

### Method Selection Guide:

- **Forward/Backward**: Use at boundaries or when function evaluations are expensive on one side
- **Central O(h²)**: Default choice for most applications
- **Central O(h⁴)**: When higher accuracy is needed and function is smooth
- **Richardson**: For maximum accuracy with smooth functions
- **Complex-step**: For analytic functions when machine precision is required

The rustlab-numerical differentiation module provides robust, efficient, and mathematically sound tools for computing derivatives in scientific and engineering applications.